### Incremental First-visit MC PE on OpenAI Gym BlackJack-V0. 


In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import numpy as np
import collections
import numpy as np
import matplotlib.pyplot as plt
import warnings
%matplotlib inline

In [2]:
import gym
from gym import spaces
from gym.utils import seeding
import numpy as np

def cmp(a, b):
    return float(a > b) - float(a < b)

# 1 = Ace, 2-10 = Number cards, Jack/Queen/King = 10
deck = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10]


def draw_card(np_random):
    return int(np_random.choice(deck))


def draw_hand(np_random):
    return [draw_card(np_random), draw_card(np_random)]


def usable_ace(hand):  # Does this hand have a usable ace?
    return 1 in hand and sum(hand) + 10 <= 21


def sum_hand(hand):  # Return current hand total
    if usable_ace(hand):
        return sum(hand) + 10
    return sum(hand)


def is_bust(hand):  # Is this hand a bust?
    return sum_hand(hand) > 21


def score(hand):  # What is the score of this hand (0 if bust)
    return 0 if is_bust(hand) else sum_hand(hand)


def is_natural(hand):  # Is this hand a natural blackjack?
    return sorted(hand) == [1, 10]



In [3]:
# BlackJack-V0 Enviroment

class BlackjackEnv(gym.Env):
    """Simple blackjack environment
    Blackjack is a card game where the goal is to obtain cards that sum to as
    near as possible to 21 without going over.  They're playing against a fixed
    dealer.
    Face cards (Jack, Queen, King) have point value 10.
    Aces can either count as 11 or 1, and it's called 'usable' at 11.
    This game is placed with an infinite deck (or with replacement).
    The game starts with each (player and dealer) having one face up and one
    face down card.
    The player can request additional cards (hit=1) until they decide to stop
    (stick=0) or exceed 21 (bust).
    After the player sticks, the dealer reveals their facedown card, and draws
    until their sum is 17 or greater.  If the dealer goes bust the player wins.
    If neither player nor dealer busts, the outcome (win, lose, draw) is
    decided by whose sum is closer to 21.  The reward for winning is +1,
    drawing is 0, and losing is -1.
    The observation of a 3-tuple of: the players current sum,
    the dealer's one showing card (1-10 where 1 is ace),
    and whether or not the player holds a usable ace (0 or 1).
    This environment corresponds to the version of the blackjack problem
    described in Example 5.1 in Reinforcement Learning: An Introduction
    by Sutton and Barto.
    http://incompleteideas.net/book/the-book-2nd.html
    """
    def __init__(self, natural=False):
        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Tuple((
            spaces.Discrete(32),
            spaces.Discrete(11),
            spaces.Discrete(2)))
        self.seed()

        # Flag to payout 1.5 on a "natural" blackjack win, like casino rules
        # Ref: http://www.bicyclecards.com/how-to-play/blackjack/
        self.natural = natural
        # Start the first game
        self.reset()

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        assert self.action_space.contains(action)
        if action:  # hit: add a card to players hand and return
            self.player.append(draw_card(self.np_random))
            if is_bust(self.player):
                done = True
                reward = -1
            else:
                done = False
                reward = 0
        else:  # stick: play out the dealers hand, and score
            done = True
            while sum_hand(self.dealer) < 17:
                self.dealer.append(draw_card(self.np_random))
            reward = cmp(score(self.player), score(self.dealer))
            if self.natural and is_natural(self.player) and reward == 1:
                reward = 1.5
        return self._get_obs(), reward, done, {}

    def _get_obs(self):
        return (sum_hand(self.player), self.dealer[0], usable_ace(self.player))

    def reset(self):
        self.dealer = draw_hand(self.np_random)
        self.player = draw_hand(self.np_random)
        return self._get_obs()

In [4]:
def sample_policy(observation):
    score, dealer_score, usable_ace = observation
    return 0 if score >= 20 else 1

In [5]:
def generate_episode_black_jack(env):
    
    states, actions, rewards = [], [], []

    state = env.reset()
    
    while True:

        states.append(state)   
        
        action = sample_policy(state)
        actions.append(action)

        state, reward, done, info = env.step(action)
        rewards.append(reward)
        

        if done:
             break
                
    return states, actions, rewards

In [6]:
def first_visit_mc_prediction(env, n_episodes):
    Q = defaultdict(float)
    N = defaultdict(int)
    gamma = 1.0
    
    for i in range(n_episodes):
        states, i, rewards = generate_episode_black_jack(env)
        returns = 0

        for t in range(len(states) - 1, -1, -1):
            R = rewards[t]
            S = states[t]
            
            returns = returns * gamma + R
            if S not in states[:t]:
                N[S] += 1
                Q[S] += (returns - Q[S]) / N[S]
    
    return Q

In [7]:
env = BlackjackEnv()
Q = first_visit_mc_prediction(env, n_episodes=10000)

### Policy Iteration and incremental first-visit MC to solve the following GridWorld environment.  


In [22]:
# maze example

class GridWorld:
    """ Grid World environment
            there are four actions (left, right, up, and down) to move an agent
            In a grid, if it reaches a goal, it get 30 points of reward.
            If it falls in a hole or moves out of the grid world, it gets -5.
            Each step costs -1 point. 

        to test GridWorld, run the following sample codes:

            env = GridWorld('grid.txt')

            env.print_map()
            print [2,3], env.check_state([2,3])
            print [0,0], env.check_state([0,0])
            print [3,4], env.check_state([3,4])
            print [10,3], env.check_state([10,3])

            env.init([0,0])
            print env.next(1)  # right
            print env.next(3)  # down
            print env.next(0)  # left
            print env.next(2)  # up
            print env.next(2)  # up

        Parameters
        ==========
        _map        ndarray
                    string array read from a file input
        _size       1d array
                    the size of _map in ndarray
        goal_pos    tuple
                    the index for the goal location
        _actions    list
                    list of actions for 4 actions
        _s          1d array
                    current state
    """
    def __init__(self, fn):
        # read a map from a file
        self._map = self.read_map(fn)
        self._size = np.asarray(self._map.shape)
        self.goal_pos = np.where(self._map == 'G')

        # definition of actions (left, right, up, and down repectively)
        self._actions = [[0, -1], [0, 1], [-1, 0], [1, 0]]
        self._s = None

    def get_cur_state(self):
        return self._s

    def get_size(self):
        return self._size

    def read_map(self, fn):
        grid = []
        with open(fn) as f:
            for line in f:
                grid.append(list(line.strip()))
        return np.asarray(grid)

    def print_map(self):
        print( self._map )

    def check_state(self, s):
        if isinstance(s, collections.Iterable) and len(s) == 2:
            if s[0] < 0 or s[1] < 0 or\
               s[0] >= self._size[0] or s[1] >= self._size[1]:
                   return 'N'
            return self._map[tuple(s)].upper()
        else:
            return 'F'  # wrong input

    def init(self, state=None):
        if state is None:
            s = [0, 0]
        else:
            s = state
        if self.check_state(s) == 'O':
            self._s = np.asarray(s)
        else:
            raise ValueError("Invalid state for init")

    def next(self, a):
        s1 = self._s + self._actions[a]
        print(self._s,"in next")
        # state transition
        curr = self.check_state(s1)
        
        if curr == 'H' or curr == 'N':
            return -5
        elif curr == 'F':
            warnings.warn("invalid state " + str(s1))
            return -5
        elif curr == 'G':
            self._s = s1
            return 30
        else:
            self._s = s1
            return -1
        
    def is_goal(self):
        return self.check_state(self._s) == 'G'
            
    def get_actions(self):
        return self._actions

In [23]:
env_MC = GridWorld("grid.txt")
env_MC.print_map()

[['O' 'O' 'O' 'H' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'H' 'O' 'O' 'H' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'H' 'O' 'O']
 ['O' 'O' 'O' 'O' 'H' 'H' 'H' 'O' 'O']
 ['O' 'O' 'H' 'O' 'O' 'O' 'O' 'O' 'H']
 ['O' 'O' 'H' 'O' 'O' 'O' 'G' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']]


In [24]:
# top-left to (0,0)
def coord_convert(s, sz):
    return [s[1], sz[0]-s[0]-1]

In [25]:
def policy_fn(epsilon, s, nA, Q):
    a = np.random.randint(nA)
    return a

In [26]:
def generate_episode_grid(env_MC_grid, nA, Q):
    
    states, actions, rewards = [], [], []

    env_MC_grid.init()
    state = env_MC_grid.get_cur_state()
    while True:
        states.append(tuple(state))
        
        action = policy_fn(0.1, state, nA, Q)
        actions.append(action)
        reward= env_MC_grid.next(action)
        rewards.append(reward)
        state = env_MC_grid.get_cur_state()

        if env_MC_grid.is_goal():
             break
                
    return states, actions, rewards

In [27]:
def first_visit_mc_prediction_grid(env_MC_grid, n_episodes):
    #Q = defaultdict(lambda: np.zeros(len(env_MC_grid.get_actions())))
    nA = len(env_MC_grid.get_actions())
#     Q = np.zeros((env_MC_grid.get_size()[0], env_MC_grid.get_size()[1], nA))
#     N = np.zeros((env_MC_grid.get_size()[0], env_MC_grid.get_size()[1], nA))
    Q = defaultdict(float)
    N = defaultdict(int)
    gamma = 1.0

    
    for i in range(n_episodes):
        states, i, rewards = generate_episode_grid(env_MC_grid, len(env_MC_grid.get_actions()), Q)

        returns = 0
        for t in range(len(states) - 1, -1, -1):
            R = rewards[t]
            S = states[t]
            returns = returns * gamma + R
            a
            if S not in states[:t]:
                N[S] += 1
                Q[S] += (returns - Q[S]) / N[S]
    
    return Q

In [28]:
env_MC_grid = GridWorld("grid.txt")
grid_Q = first_visit_mc_prediction_grid(env_MC_grid, 100)
print(grid_Q)

[0 0] in next
[0 0] in next
[0 0] in next
[0 1] in next
[1 1] in next
[0 1] in next
[1 1] in next
[0 1] in next
[0 2] in next
[0 1] in next
[0 1] in next
[1 1] in next
[2 1] in next
[3 1] in next
[2 1] in next
[2 0] in next
[2 1] in next
[3 1] in next
[3 0] in next
[3 0] in next
[2 0] in next
[1 0] in next
[1 0] in next
[1 1] in next
[2 1] in next
[2 0] in next
[3 0] in next
[4 0] in next
[3 0] in next
[2 0] in next
[2 0] in next
[2 0] in next
[3 0] in next
[3 1] in next
[3 2] in next
[3 3] in next
[4 3] in next
[5 3] in next
[6 3] in next
[6 3] in next
[6 4] in next
[6 3] in next
[5 3] in next
[5 4] in next
[4 4] in next
[4 5] in next
[4 5] in next
[5 5] in next
[5 4] in next
[5 3] in next
[4 3] in next
[3 3] in next
[2 3] in next
[3 3] in next
[3 2] in next
[3 2] in next
[2 2] in next
[2 3] in next
[2 2] in next
[1 2] in next
[2 2] in next
[1 2] in next
[0 2] in next
[0 2] in next
[0 2] in next
[0 1] in next
[0 1] in next
[0 2] in next
[0 2] in next
[0 2] in next
[0 2] in next
[0 2] 

[6 4] in next
[6 4] in next
[5 4] in next
[6 4] in next
[6 4] in next
[6 4] in next
[6 5] in next
[6 4] in next
[6 4] in next
[6 5] in next
[6 6] in next
[0 0] in next
[0 1] in next
[0 2] in next
[0 2] in next
[1 2] in next
[2 2] in next
[1 2] in next
[0 2] in next
[1 2] in next
[2 2] in next
[2 3] in next
[2 2] in next
[1 2] in next
[1 2] in next
[0 2] in next
[0 2] in next
[0 2] in next
[0 2] in next
[0 1] in next
[0 2] in next
[0 2] in next
[1 2] in next
[1 2] in next
[1 1] in next
[0 1] in next
[0 0] in next
[0 0] in next
[0 0] in next
[1 0] in next
[1 0] in next
[0 0] in next
[0 1] in next
[0 0] in next
[0 0] in next
[0 0] in next
[0 0] in next
[0 1] in next
[1 1] in next
[0 1] in next
[0 1] in next
[1 1] in next
[1 0] in next
[0 0] in next
[0 0] in next
[0 0] in next
[1 0] in next
[1 0] in next
[1 1] in next
[1 2] in next
[1 2] in next
[2 2] in next
[1 2] in next
[1 2] in next
[0 2] in next
[1 2] in next
[0 2] in next
[0 1] in next
[0 0] in next
[0 0] in next
[1 0] in next
[0 0] 

[6 3] in next
[6 3] in next
[5 3] in next
[5 4] in next
[6 4] in next
[5 4] in next
[5 3] in next
[6 3] in next
[6 3] in next
[6 2] in next
[6 2] in next
[6 3] in next
[5 3] in next
[5 4] in next
[5 5] in next
[0 0] in next
[0 0] in next
[0 0] in next
[0 1] in next
[1 1] in next
[1 2] in next
[1 1] in next
[1 2] in next
[1 1] in next
[1 0] in next
[1 0] in next
[1 0] in next
[1 1] in next
[0 1] in next
[1 1] in next
[1 0] in next
[2 0] in next
[1 0] in next
[2 0] in next
[3 0] in next
[4 0] in next
[3 0] in next
[2 0] in next
[3 0] in next
[3 1] in next
[2 1] in next
[2 0] in next
[3 0] in next
[4 0] in next
[3 0] in next
[4 0] in next
[5 0] in next
[6 0] in next
[5 0] in next
[5 1] in next
[6 1] in next
[6 0] in next
[6 0] in next
[6 0] in next
[5 0] in next
[5 1] in next
[5 1] in next
[6 1] in next
[6 0] in next
[6 1] in next
[6 2] in next
[6 2] in next
[6 2] in next
[6 2] in next
[6 2] in next
[6 2] in next
[6 2] in next
[6 2] in next
[6 1] in next
[6 2] in next
[6 2] in next
[6 2] 

[5 1] in next
[5 0] in next
[5 1] in next
[5 0] in next
[5 0] in next
[4 0] in next
[4 0] in next
[4 0] in next
[3 0] in next
[4 0] in next
[4 1] in next
[5 1] in next
[5 0] in next
[4 0] in next
[3 0] in next
[2 0] in next
[1 0] in next
[0 0] in next
[0 0] in next
[0 1] in next
[0 2] in next
[1 2] in next
[2 2] in next
[2 3] in next
[2 3] in next
[2 4] in next
[2 4] in next
[2 3] in next
[2 4] in next
[1 4] in next
[2 4] in next
[1 4] in next
[1 5] in next
[2 5] in next
[2 5] in next
[2 5] in next
[2 4] in next
[2 4] in next
[2 5] in next
[2 5] in next
[2 5] in next
[2 4] in next
[2 5] in next
[2 5] in next
[2 5] in next
[1 5] in next
[0 5] in next
[0 4] in next
[0 4] in next
[0 5] in next
[1 5] in next
[1 5] in next
[2 5] in next
[2 5] in next
[2 4] in next
[2 3] in next
[2 3] in next
[2 2] in next
[3 2] in next
[3 2] in next
[2 2] in next
[1 2] in next
[0 2] in next
[1 2] in next
[1 1] in next
[2 1] in next
[3 1] in next
[2 1] in next
[3 1] in next
[3 0] in next
[3 1] in next
[3 2] 

[2 0] in next
[3 0] in next
[3 1] in next
[4 1] in next
[5 1] in next
[6 1] in next
[6 0] in next
[6 1] in next
[5 1] in next
[4 1] in next
[4 1] in next
[5 1] in next
[4 1] in next
[5 1] in next
[6 1] in next
[6 0] in next
[5 0] in next
[4 0] in next
[3 0] in next
[3 0] in next
[2 0] in next
[2 1] in next
[3 1] in next
[2 1] in next
[3 1] in next
[3 2] in next
[3 3] in next
[3 2] in next
[3 3] in next
[4 3] in next
[4 4] in next
[4 5] in next
[4 5] in next
[5 5] in next
[5 4] in next
[6 4] in next
[5 4] in next
[6 4] in next
[6 4] in next
[6 4] in next
[6 3] in next
[6 3] in next
[6 4] in next
[6 5] in next
[6 4] in next
[5 4] in next
[5 3] in next
[5 3] in next
[5 4] in next
[5 5] in next
[4 5] in next
[4 5] in next
[4 4] in next
[4 3] in next
[4 4] in next
[4 4] in next
[5 4] in next
[4 4] in next
[4 4] in next
[4 5] in next
[4 4] in next
[4 5] in next
[4 5] in next
[4 4] in next
[4 3] in next
[4 3] in next
[4 3] in next
[4 4] in next
[5 4] in next
[4 4] in next
[4 4] in next
[4 3] 

...

[6 2] in next
[6 2] in next
[6 1] in next
[6 2] in next
[6 2] in next
[6 2] in next
[6 2] in next
[6 1] in next
[6 0] in next
[5 0] in next
[5 1] in next
[4 1] in next
[4 1] in next
[4 1] in next
[4 1] in next
[3 1] in next
[4 1] in next
[4 1] in next
[4 1] in next
[4 1] in next
[4 0] in next
[5 0] in next
[4 0] in next
[3 0] in next
[2 0] in next
[1 0] in next
[2 0] in next
[3 0] in next
[4 0] in next
[5 0] in next
[5 1] in next
[5 0] in next
[5 1] in next
[5 0] in next
[5 1] in next
[5 1] in next
[5 0] in next
[5 0] in next
[4 0] in next
[3 0] in next
[4 0] in next
[3 0] in next
[2 0] in next
[1 0] in next
[0 0] in next
[0 0] in next
[0 0] in next
[1 0] in next
[2 0] in next
[1 0] in next
[1 0] in next
[0 0] in next
[0 1] in next
[0 0] in next
[1 0] in next
[1 1] in next
[2 1] in next
[2 0] in next
[2 0] in next
[2 0] in next
[2 1] in next
[2 0] in next
[3 0] in next
[3 0] in next
[3 1] in next
[4 1] in next
[5 1] in next
[5 1] in next
[4 1] in next
[3 1] in next
[3 0] in next
[4 0] 

[2 1] in next
[2 0] in next
[2 1] in next
[2 2] in next
[2 1] in next
[3 1] in next
[3 2] in next
[3 1] in next
[4 1] in next
[4 1] in next
[5 1] in next
[4 1] in next
[4 0] in next
[4 1] in next
[4 0] in next
[4 0] in next
[4 0] in next
[5 0] in next
[6 0] in next
[6 0] in next
[6 0] in next
[6 0] in next
[6 0] in next
[5 0] in next
[6 0] in next
[5 0] in next
[5 0] in next
[5 1] in next
[4 1] in next
[4 0] in next
[3 0] in next
[2 0] in next
[2 0] in next
[3 0] in next
[2 0] in next
[2 1] in next
[2 2] in next
[2 1] in next
[1 1] in next
[1 2] in next
[2 2] in next
[2 1] in next
[1 1] in next
[1 2] in next
[1 2] in next
[1 1] in next
[1 2] in next
[1 2] in next
[0 2] in next
[0 2] in next
[0 1] in next
[1 1] in next
[1 0] in next
[1 0] in next
[1 0] in next
[1 0] in next
[1 1] in next
[0 1] in next
[1 1] in next
[1 2] in next
[1 2] in next
[0 2] in next
[1 2] in next
[2 2] in next
[2 3] in next
[2 2] in next
[2 1] in next
[2 2] in next
[1 2] in next
[1 2] in next
[2 2] in next
[2 1] 

[4 1] in next
[5 1] in next
[5 1] in next
[5 1] in next
[5 0] in next
[6 0] in next
[6 1] in next
[6 1] in next
[6 1] in next
[6 2] in next
[6 2] in next
[6 3] in next
[5 3] in next
[4 3] in next
[5 3] in next
[6 3] in next
[6 4] in next
[6 4] in next
[6 5] in next
[6 6] in next
[0 0] in next
[0 1] in next
[0 1] in next
[1 1] in next
[1 0] in next
[2 0] in next
[2 1] in next
[3 1] in next
[4 1] in next
[4 0] in next
[4 0] in next
[4 0] in next
[4 0] in next
[4 1] in next
[4 1] in next
[4 1] in next
[4 0] in next
[4 1] in next
[3 1] in next
[3 2] in next
[3 2] in next
[2 2] in next
[3 2] in next
[3 1] in next
[2 1] in next
[2 0] in next
[3 0] in next
[2 0] in next
[2 0] in next
[2 1] in next
[3 1] in next
[4 1] in next
[3 1] in next
[3 2] in next
[3 1] in next
[4 1] in next
[4 1] in next
[3 1] in next
[3 2] in next
[3 2] in next
[3 2] in next
[3 3] in next
[4 3] in next
[4 3] in next
[5 3] in next
[6 3] in next
[5 3] in next
[5 4] in next
[6 4] in next
[6 5] in next
[6 5] in next
[6 4] 

[3 8] in next
[2 8] in next
[2 7] in next
[3 7] in next
[3 8] in next
[3 8] in next
[3 8] in next
[2 8] in next
[2 7] in next
[1 7] in next
[2 7] in next
[2 7] in next
[3 7] in next
[3 8] in next
[3 8] in next
[2 8] in next
[2 8] in next
[2 8] in next
[3 8] in next
[3 8] in next
[3 8] in next
[3 7] in next
[4 7] in next
[4 7] in next
[3 7] in next
[3 7] in next
[3 7] in next
[4 7] in next
[4 7] in next
[5 7] in next
[0 0] in next
[1 0] in next
[1 1] in next
[2 1] in next
[1 1] in next
[1 2] in next
[1 1] in next
[1 2] in next
[2 2] in next
[1 2] in next
[0 2] in next
[0 1] in next
[0 2] in next
[0 2] in next
[0 2] in next
[1 2] in next
[1 2] in next
[1 2] in next
[1 1] in next
[0 1] in next
[0 0] in next
[0 0] in next
[0 0] in next
[1 0] in next
[2 0] in next
[3 0] in next
[3 0] in next
[4 0] in next
[5 0] in next
[5 0] in next
[6 0] in next
[6 0] in next
[6 0] in next
[6 0] in next
[6 0] in next
[5 0] in next
[5 0] in next
[4 0] in next
[5 0] in next
[4 0] in next
[4 1] in next
[4 1] 

KeyboardInterrupt: 

### Grid World Implementation for Policy iteration using DP:

In [111]:
class GridAgent:
    
    def __init__(self, env):
        self.env = env
        self.size = np.prod(env.get_size())
        self.n_a = len(env.get_actions())
        self.policy = np.zeros([self.env.get_size()[0], self.env.get_size()[1], self.n_a])
        self.policy[:, :, 0] = 1
        self.V = np.zeros((self.env.get_size()[0], self.env.get_size()[1]))
        self.gamma = 0.9
        self.theta = 0.001
 
    def policy_eval(self):
        while True:
            delta = 0
            for i in range(self.env.get_size()[0]):
                for j in range(self.env.get_size()[1]):
                    v = self.V[i][j]
                    if self.env.check_state([i, j]) != 'O':
                        continue
                    self.env.init([i,j])
                    reward = self.env.next(np.argmax(self.policy[i][j]))
                    next_state = self.env.get_cur_state()

                    v = (reward + self.gamma * self.V[next_state[0]] [next_state[1]])

                    delta = max(delta, np.abs(v - self.V[i,j]))
                    self.V[i,j] = v
            
            if delta < self.theta:
                break
    
    def policy_improvement(self, start):
        #V = self.policy_eval()
        env_grid.init(start)
        cnt = 0
        while True:
            self.policy_eval()

            policy_stable = True

            for i in range(self.env.get_size()[0]):
                for j in range(self.env.get_size()[1]):
                    if self.env.check_state([i, j]) != 'O':
                        continue
                    old_action = np.argmax(self.policy[i, j, :])
                    action = self.next_action(i,j)
                    best_action = np.argmax(action)

                    if old_action != best_action:
                        policy_stable = False
                    self.policy[i, j, :] = np.eye(self.n_a)[best_action]
                    
            if policy_stable:
                return self.policy, self.V

            
    
    
    def next_action(self,i,j):
        actions = np.zeros(self.n_a)
        
        for a in range(self.n_a):
            self.env.init([i,j])
            reward = self.env.next(a)
            next_state = self.env.get_cur_state()
            actions[a] += (reward + self.gamma * self.V[next_state[0]] [next_state[1]])
        return actions
            
        


        

In [113]:
env_grid = GridWorld("grid.txt")
gridAgent = GridAgent(env_grid)
start = [0,0]
policy, values = gridAgent.policy_improvement(start)

In [114]:
print(policy, values)

[[[0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [1. 0. 0. 